In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Step 1, 2 데이터 다운로드 및 읽어오기

In [2]:
import glob
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

txt_file_path = './lyricist/data/lyrics/*.txt'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


# Step 3. 데이터 정제

In [3]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    new_sentence = preprocess_sentence(sentence)
    if len(new_sentence) == 0: continue
    if new_sentence[-1] == ":": continue
    
    # if new_sentence.count(" ") <= 13:
    # if len(new_sentence.split()) <= 15: # without corpus having more than 15 token
    #    corpus.append(new_sentence)
    corpus.append(preprocess_sentence(sentence))
    
        
corpus[:10]

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> here without that look upon your face i should have changed that fucking lock <end>']

In [5]:
new_corpus = []
for i in range(len(corpus) - 1):
    if len(corpus[i].split()) <= 13: # <Start> & <End>
        new_corpus.append(corpus[i])

In [6]:
len(corpus)

187088

In [7]:
len(new_corpus)

153375

In [8]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(new_corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(new_corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre', maxlen = 15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   0    0    0 ...   52  705    3]
 [   0    0    0 ...   52 5859    3]
 [   0    0    0 ...  259    7    3]
 ...
 [   0    0    0 ...    1  470    3]
 [   0    0    0 ...  887    5    3]
 [   0    0    0 ... 1895   60    3]] <keras_preprocessing.text.Tokenizer object at 0x7fbd057ea190>


In [9]:
tensor

array([[   0,    0,    0, ...,   52,  705,    3],
       [   0,    0,    0, ...,   52, 5859,    3],
       [   0,    0,    0, ...,  259,    7,    3],
       ...,
       [   0,    0,    0, ...,    1,  470,    3],
       [   0,    0,    0, ...,  887,    5,    3],
       [   0,    0,    0, ..., 1895,   60,    3]], dtype=int32)

In [10]:
len(tensor[0])

15

In [11]:
tokenizer

In [12]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  0   0   0   0   0   0   0   0   2  72 254   4  52 705]
[  0   0   0   0   0   0   0   2  72 254   4  52 705   3]


# Step 4. 평가 데이터셋 분리

In [13]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, shuffle = True)

In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [15]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (122700, 14)
Target Train: (122700, 14)


# Step 5. 인공지능 만들기

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [17]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-3.6643352e-04, -1.1438113e-04, -1.8336905e-05, ...,
         -1.4537302e-04,  1.3057933e-04, -1.2463293e-04],
        [-8.9791755e-04, -3.1397751e-04, -5.3519409e-05, ...,
         -3.2656293e-04,  3.2892224e-04, -2.9461391e-04],
        [-1.4783415e-03, -5.7069195e-04, -1.0073532e-04, ...,
         -4.9852324e-04,  5.5792753e-04, -4.7033458e-04],
        ...,
        [-1.1056314e-03, -1.5304888e-04, -8.1393501e-04, ...,
         -2.6575386e-04,  5.1886658e-04, -3.9156707e-04],
        [-7.1244483e-04,  2.8571390e-05, -1.1897177e-03, ...,
         -4.2006979e-04,  5.2934175e-04, -1.9000357e-04],
        [-2.6484576e-04, -4.7747038e-05, -1.3833991e-03, ...,
         -5.4268283e-04,  6.2855869e-04,  1.3353548e-04]],

       [[-3.6643352e-04, -1.1438113e-04, -1.8336905e-05, ...,
         -1.4537302e-04,  1.3057933e-04, -1.2463293e-04],
        [-8.9791755e-04, -3.1397751e-04, -5.3519409e-05, ...,
         -3.2656293e-04, 

In [18]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
479/479 [==============================] - 59s 123ms/step - loss: 3.3158
Epoch 2/30
479/479 [==============================] - 58s 122ms/step - loss: 2.8960
Epoch 3/30
479/479 [==============================] - 62s 129ms/step - loss: 2.8480
Epoch 4/30
479/479 [==============================] - 59s 123ms/step - loss: 2.7408
Epoch 5/30
479/479 [==============================] - 60s 126ms/step - loss: 2.6654
Epoch 6/30
479/479 [==============================] - 59s 123ms/step - loss: 2.6344
Epoch 7/30
479/479 [==============================] - 60s 125ms/step - loss: 2.5662
Epoch 8/30
479/479 [==============================] - 60s 125ms/step - loss: 2.5049
Epoch 9/30
479/479 [==============================] - 61s 128ms/step - loss: 2.4447
Epoch 10/30
479/479 [==============================] - 61s 127ms/step - loss: 2.3938
Epoch 11/30
479/479 [==============================] - 59s 124ms/step - loss: 2.3452
Epoch 12/30
479/479 [==============================] - 59s 124ms/step - lo

In [20]:
# 모델 검증

results = model.evaluate(enc_val, dec_val, verbose = 2)
print(results)

959/959 - 8s - loss: 2.2979
2.297917366027832


In [21]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len = 20)

'<start> i tag royal tag circles unlock ripped tweetle tag circles ripped tweetle tag ripped tweetle tag circles ripped circles '

# 회고

### - Pre로 진행한 뒤
    전반적인 모델의 성능이 저하되는 것을 확인하였다.
    LSTM을 활용한 모델이어서 pre-padding의 경우 성능이 더 좋을 줄 알았는데, 큰 차이는 없었지만, 성능이 조금 저하되는 것을 확인하니 더 혼란스럽다..
    
    